<a href="https://colab.research.google.com/github/cowiekw/bio-project/blob/master/drugindications_longdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Path: "/Users/katecowie/Documents/fall2020/disease coherence/drug-indications/analysis/lowercase_drug_indications_data.xlsx"

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
olddf = pd.read_excel("lowercase_drug_indications_data.xlsx")
olddf.astype(str)
olddf.head()

,rxcui_in,drug_desc,icd9,indication,possible_label_use
0,85248,alosetron,789.00,Abdominal pain,1
1,4440,flavoxate,789.09,Abdominal pain; other specified site; multiple...,1
2,1223,atropine,789.00,Abdominal pain; unspecified site,1
3,1225,hyoscyamine sulfate,789.00,Abdominal pain; unspecified site,0
4,1851,butylscopolamine bromide,789.00,Abdominal pain; unspecified site,1


In [ ]:
drug_ind = olddf[["indication","drug_desc"]]
drug_ind.head()

,indication,drug_desc
0,Abdominal pain,alosetron
1,Abdominal pain; other specified site; multiple...,flavoxate
2,Abdominal pain; unspecified site,atropine
3,Abdominal pain; unspecified site,hyoscyamine sulfate
4,Abdominal pain; unspecified site,butylscopolamine bromide


### Subset the dataframe to get indications and icd9 codes. 

In [ ]:
icd_ind = olddf[["indication","icd9"]]
icd_ind =icd_ind.astype(str)

In [ ]:
icddf = icd_ind.groupby('indication').agg(lambda x: ','.join(x.unique())).reset_index()
icddf.head()

,indication,icd9
0,Abdominal pain,789.0
1,Abdominal pain; other specified site; multiple...,789.09
2,Abdominal pain; unspecified site,789.0
3,Abnormal feces,787.7
4,Abnormal involuntary movements,781.0


### Group by each unique indication and all of the drug names.

In [ ]:
newdf = drug_ind.groupby('indication').agg(lambda x: ','.join(x.unique())).reset_index()
newdf.head()

,indication,drug_desc
0,Abdominal pain,alosetron
1,Abdominal pain; other specified site; multiple...,flavoxate
2,Abdominal pain; unspecified site,"atropine,hyoscyamine sulfate,butylscopolamine ..."
3,Abnormal feces,"activated charcoal,atropine,charcoal,cholestyr..."
4,Abnormal involuntary movements,"atropine,hyoscyamine sulfate,baclofen,belladon..."


In [ ]:
mergedf = icddf.merge(newdf, how='inner', on='indication')
len(mergedf)

76

## Add # of Drugs per Indication

In [ ]:
numdf = pd.read_excel("n_indicatios_2.xlsx")
numdf.tail()

,indication,n()
71,Amyloidosis NOS,2
72,Amyotrophic lateral sclerosis,3
73,Anal fissure,10
74,Anatomical narrow angle borderline glaucoma,6
75,Ancylostomiasis NOS,1


In [ ]:
df= mergedf.merge(numdf, how='left', on='indication')
df.head()

,indication,icd9,drug_desc,n()
0,Abdominal pain,789.0,alosetron,1
1,Abdominal pain; other specified site; multiple...,789.09,flavoxate,1
2,Abdominal pain; unspecified site,789.0,"atropine,hyoscyamine sulfate,butylscopolamine ...",9
3,Abnormal feces,787.7,"activated charcoal,atropine,charcoal,cholestyr...",38
4,Abnormal involuntary movements,781.0,"atropine,hyoscyamine sulfate,baclofen,belladon...",49


In [ ]:
df.to_excel("drug_indications_reformat.xlsx", index=False)

Split the drug desc column

In [ ]:
df.head()

,indication,icd9,drug_desc,n()
0,Abdominal pain,789.0,alosetron,1
1,Abdominal pain; other specified site; multiple...,789.09,flavoxate,1
2,Abdominal pain; unspecified site,789.0,"atropine,hyoscyamine sulfate,butylscopolamine ...",9
3,Abnormal feces,787.7,"activated charcoal,atropine,charcoal,cholestyr...",38
4,Abnormal involuntary movements,781.0,"atropine,hyoscyamine sulfate,baclofen,belladon...",49


In [ ]:
df=pd.concat([df,df.drug_desc.str.split(',',expand=True)],1)
df.head()

In [ ]:
longdf = df.astype(str)

## Make empty data NUll


In [ ]:
NaN = float("NaN")
longdf.replace("None", NaN, inplace=True)
longdf.head()

,indication,icd9,drug_desc,n(),0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113
0,Abdominal pain,789.0,alosetron,1,alosetron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abdominal pain; other specified site; multiple...,789.09,flavoxate,1,flavoxate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abdominal pain; unspecified site,789.0,"atropine,hyoscyamine sulfate,butylscopolamine ...",9,atropine,hyoscyamine sulfate,butylscopolamine bromide,papaverine,alosetron,tegaserod,hyoscyamine,pentosan polysulfate,hyoscyamine hydrobromide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abnormal feces,787.7,"activated charcoal,atropine,charcoal,cholestyr...",38,activated charcoal,atropine,charcoal,cholestyramine resin,clonidine,colestipol,diphenoxylate,epinephrine,furazolidone,loperamide,methylcellulose,octreotide,opium,potassium,psyllium,salicylic acid,sulfamethoxazole / trimethoprim,glucomannan,attapulgite,bismuth subsalicylate,calcium polycarbophil,difenoxin,nitazoxanide,paregoric,sodium bicarbonate,diphenoxylate hydrochloride,loperamide hydrochloride,alosetron,malt soup extract,colestipol hydrochloride,colesevelam hydrochloride,colesevelam,clonidine hydrochloride,atropine sulfate,loperamide / simethicone,difenoxin hydrochloride,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abnormal involuntary movements,781.0,"atropine,hyoscyamine sulfate,baclofen,belladon...",49,atropine,hyoscyamine sulfate,baclofen,belladonna alkaloids,diphenhydramine hydrochloride,butylscopolamine bromide,carisoprodol,chlormezanone,chlorphenesin,chlorzoxazone,clonazepam,clonidine,dantrolene,dantrolene sodium,dextromethorphan,diazepam,dicyclomine,diphenhydramine,epinephrine,ibuprofen,meprobamate,methocarbamol,nifedipine,orphenadrine,orphenadrine citrate,papaverine,prazepam,primidone,propantheline,propranolol,tolazoline,cyclobenzaprine,oxybutynin,sertraline,cyclobenzaprine hydrochloride,oxybutynin chloride,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
longdf.to_excel("drug_ind_longdf.xlsx", index=False)

### Change the data so that each column corresponds to a drug. 

insert column "flavoxate"

Remove columns that have no data

In [ ]:
longdf.dropna(axis=1, how= 'all', thresh=None, subset=None, inplace=True)
longdf.tail()

## Split the columns

In [ ]:
df['drug1'], df['drug2'] = df['drug_desc'].str.split(',', 1).str

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.
